In [1]:
# Temporary for correct and eacy insertion of tables
hp_res = {}
bt_res = {}

# Imports

In [2]:
import sys
sys.path.append('/home/si-hj/Desktop/medsegnet')
sys.path.append('/home/si-hj/Desktop')
import numpy as np
from ast import mod
import pathlib
import numpy as np
from omegaconf import DictConfig, OmegaConf
from typing import Dict, Tuple
import torchio as tio
import torch
import matplotlib.pyplot as plt
import os
import data
from data.datasets import MedicalDecathlonDataset, ModalitiesDataset
from utils.assertions import ensure

from pathlib import Path
from time import time
import nibabel as nib
from utils.metrics import dice_coefficient, dice_coefficient_classes, accuracy_score
from utils.utils import setup_seed
from hydra.utils import instantiate
import os
from data.datasets import MedicalDecathlonDataset
import torch.nn.functional as F
from tqdm import tqdm
from torch.utils.data import DataLoader

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# <span style="color:#fcba03">Data simulation</span>:

### Generel code Blocks

In [3]:
def model_params(model_dir):
	if not model_dir.is_dir():
		raise FileNotFoundError(f"Model directory not found: {model_dir}")
	
	model_path = f"{model_dir}/best_model.pth"
	cfg = OmegaConf.load(f"{model_dir}/config.yaml")
	
	setup_seed(cfg.seed)
	
	if not isinstance(cfg, DictConfig):
		raise TypeError("cfg must be a DictConfig.")

	return model_path, cfg

def pp_list(lst):
	return [f"{dice:.4f}" for dice in lst]

def print_results(dsc_results_per_scale, datasets = None, lowest_dice_plt = False, latex = False):
	dice_scales = []
	class_scales = []
	latex_row = ""
	for scale, results_list in dsc_results_per_scale.items():
		results = np.mean(results_list, axis=1)
		mean_pr_class = np.mean(results_list, axis=0)
		class_scales.append(mean_pr_class)
		std_pr_class = np.std(results_list, axis=0)

		mean_pr_class = [f"{dice:.4f}" for dice in mean_pr_class]
		std_pr_class = [f"{std:.4f}" for std in std_pr_class]

		lowest_idx = None
		lowest_dice = float('inf')

		if datasets is not None and lowest_dice_plt:
			for i, dice in enumerate(results):
				
				if 0.87 < dice and dice < lowest_dice:
					lowest_dice = dice
					lowest_idx = i

		if datasets is not None:
			max_idx, max_dice = datasets[scale] @ np.argmax(results), np.max(results)
			min_idx, min_dice = datasets[scale] @ np.argmin(results), np.min(results)

		
		mean_dice = np.mean(results)
		dice_scales.append(mean_dice)
		std_dice = np.std(results)
		latex_row += f"{mean_dice:.4f} \pm {std_dice:.4f} & "
		str = f"Scale {scale}) Mean Dice: {mean_dice:.4f}\t Std: {std_dice:.4f}\t Mean Pr. Class Dice: {mean_pr_class}\t Std Pr. Class: {std_pr_class}"
		if datasets:
			if lowest_idx is not None and lowest_dice_plt:
				print(f"\tChosen image: {datasets[scale] @ lowest_idx} = {lowest_dice:.4f} DSC")
			print(str + f"\t\tMax Dice({max_idx}): {max_dice:.4f}\t Min Dice({min_idx}): {min_dice:.4f}" if datasets is not None else str)
		else:
			print(str)
	print(f"Mean Dice across all scales: {np.mean(dice_scales):.4f} \t Std: {np.std(dice_scales):.4f}")
	print(f"Mean Dice across all scales of classess: {pp_list(np.mean(class_scales, axis=0))} \t Std: {pp_list(np.std(class_scales, axis=0))}")
	if latex:
		latex_row += f"{np.mean(dice_scales):.4f} \pm {np.std(dice_scales):.4f} \\\\"
		print(f"Latex: {latex_row}")
	
	print()

def consistent_dice(pred, label, num_classes):
	return dice_coefficient_classes(
			pred, label, num_classes, ignore_index=0
		)

def plot_img(image, pred, label):
	image = image[0, 1, :, :, :]
	pred = pred.cpu()

	max_shape = max(image.shape)
	cp = CropOrPad((max_shape, max_shape, max_shape))
	image = cp(image.unsqueeze(0)).squeeze(0)
	label = cp(label[:, :, :].unsqueeze(0)).squeeze(0)
	pred = cp(pred[:, :, :].unsqueeze(0)).squeeze(0)
	

	# Extract slices
	axial_image = np.rot90(image[:, :, image.shape[2] // 2])
	coronal_image = np.rot90(image[:, image.shape[1] // 2, :])
	sagittal_image = np.rot90(image[image.shape[0] // 2, :, :])
	
	axial_label = np.rot90(label[:, :, label.shape[2] // 2])
	coronal_label = np.rot90(label[:, label.shape[1] // 2, :])
	sagittal_label = np.rot90(label[label.shape[0] // 2, :, :])

	
	axial_pred = np.rot90(pred[:, :, pred.shape[2] // 2])
	coronal_pred = np.rot90(pred[:, pred.shape[1] // 2, :])
	sagittal_pred = np.rot90(pred[pred.shape[0] // 2, :, :])

	images = [
		axial_image, axial_label, axial_pred,
		coronal_image, coronal_label, coronal_pred,
		sagittal_image, sagittal_label, sagittal_pred
	]

	titles_x = ["Image", "Label", "Prediction"]
	titles_y = ["Axial", "Coronal", "Sagittal"]

	
	fig, axs = plt.subplots(3, 3, figsize=(9, 9))
	for i in range(3):
		for j in range(3):
			axs[i, j].imshow(images[i * 3 + j], cmap="gray")
			# axs[i, j].axis("off")
			if i == 0:
				axs[i, j].set_title(titles_x[j])
				
			if j == 0:
				label_obj = axs[i, j].set_ylabel(titles_y[i], rotation=90, labelpad=20)
				label_obj.set_verticalalignment('center')
	
	plt.tight_layout()
	plt.show()

In [4]:
_dataset_config = {
    "Task01_BrainTumour": {
        "best_models_paths": {
            "bb": "trained_models/unet3d/Task01_BrainTumour/2025-07-01_02:56:40_Brats-Baseline-UNet",
            "aug": "trained_models/unet-aug3d/Task01_BrainTumour/2025-07-01_02:56:40_Brats-Baseline-UNet-Aug", 
            "ms": "trained_models/ms-unet3d/Task01_BrainTumour/2025-07-01_02:56:40_Brats-RAREUNet-Final"    
        },
    },
    "Task04_Hippocampus": {
        "best_models_paths": {
            "bb": "trained_models/unet3d/Task04_Hippocampus/2025-07-01_02:56:37_Hippo-Baseline-UNet",
            "aug": "trained_models/unet-aug3d/Task04_Hippocampus/2025-07-01_02:56:37_Hippo-Baseline-UNet-Aug",
            "ms": "trained_models/ms-unet3d/Task04_Hippocampus/2025-07-01_02:56:37_Hippo-RAREUNet-Final"
        },
    },
    "Task05_Prostate": {
        "best_models_paths": {
            "bb": "",
            "aug": "",
            "ms": "" # "trained_models/ms-unet3d/Task05_Prostate/2025-06-14_18:28:37_test_prostate_instance/"
        },
    }
}

### DataSimulation -- Our model

In [5]:
import warnings

from networkx import center
import test
from torchio import CropOrPad
warnings.filterwarnings("ignore", category=UserWarning)

def test_lowres(model_dir_str: str, desc: str = "", classes: bool = True, plot: bool = False, num_ms_levels: int = 3):
	if model_dir_str is None or model_dir_str == "":
		print("No model directory provided.")
		return
	if not model_dir_str.startswith("/home/si-hj/Desktop/medsegnet/"):
		model_dir_str = "/home/si-hj/Desktop/medsegnet/" + model_dir_str
	model_dir = Path(model_dir_str)
	model_path, cfg = model_params(model_dir)

	scales = list(range(num_ms_levels+1))


	base_dir = Path(cfg.dataset.base_path)
	if not str(base_dir).endswith("_test1"):
		base_dir = Path(f"{base_dir}_test1")
		
	lowres_dir = base_dir / "lowres" / "downsampled"

	datasets = {}

	for scale in scales:
		img_dir = lowres_dir / f"scale{scale}" / "imagesTs"
		lbl_dir = lowres_dir / f"scale{scale}" / "labelsTs"
		img_dir_sort = sorted(os.listdir(img_dir))
		lbl_dir_sort = sorted(os.listdir(lbl_dir))
		if (cfg.dataset.name == "Task04_Hippocampus"):
			dataset = MedicalDecathlonDataset(cfg, "test", img_dir_sort, lbl_dir_sort, str(img_dir), str(lbl_dir))
		else:
			dataset = ModalitiesDataset(cfg, "test", img_dir_sort, lbl_dir_sort, str(img_dir), str(lbl_dir))
		datasets[scale] = dataset


	
	model = instantiate(cfg.architecture.path, cfg, "inference")

	checkpoint = torch.load(model_path, map_location=device)
	model_state_dict = checkpoint["model_state_dict"]
	model.load_state_dict(model_state_dict)
	model.to(device)
	model.eval()

	dsc_results_per_scale = {scale: [] for scale in scales}

	print(f"******************* TEST LowRes {desc + ' ' if desc != '' else ''}*******************")
	for scale in scales:
		with torch.no_grad():
			dataloader = DataLoader(datasets[scale], batch_size=1, shuffle=False, num_workers=cfg.training.num_workers)
			for idx, (image, label) in tqdm(enumerate(dataloader), total=len(dataloader), desc=f"Scale {scale}"):
				
				output = model.run_inference(image.to(device))
				pred = torch.argmax(output, dim=1).squeeze(0)
				label = label.squeeze(0)
				
				dsc_pr_class = consistent_dice(pred, label.to(device), cfg.dataset.num_classes)
				dsc_results_per_scale[scale].append([dsc_class.item() for dsc_class in dsc_pr_class])

				if plot and (scale == 0 and np.mean(dsc_pr_class) <= 0.3 or np.mean(dsc_pr_class) < 0.1):
					print(f"dice: {dsc_pr_class.item()}\t{datasets[scale] @ idx}")
					plot_img(image, pred, label)
						

	print(f"Results per scale of lowres:")
	print_results(dsc_results_per_scale, datasets, lowest_dice_plt=False, latex=True)
	return dsc_results_per_scale

for path, task in [
	# ("trained_models/ms-unet3d/Task01_BrainTumour/2025-06-13_18:28:57_msinstance_0dropout_msgate-instancenorm", "bt - check"),
 	# ("trained_models/ms-unet3d/Task01_BrainTumour/2025-06-13_03:34:22_msinstance_full", "bt - test")	
]:
	if not path.startswith("/home/si-hj/Desktop/medsegnet/") and path != "":
		path = "/home/si-hj/Desktop/medsegnet/" + path
	if not os.path.exists(path):
		print(f"Path does not exist: {path}")
		continue
	test_lowres(path, f"quciktest | Test runs {task} Models (Multiscale)")

# Hippocampus
print(f"{'*' * 20} HippoCampus {'*' * 20}")
hp_res["our"] = test_lowres(_dataset_config['Task04_Hippocampus']['best_models_paths'].get('ms'), "HippoCampus | Best Model (Multiscale)")

# Prostate
print(f"{'*' * 20} Prostate {'*' * 20}")
# test_lowres(_dataset_config['Task05_Prostate']['best_models_paths'].get('ms'), "Prostate | Best Model (Multiscale)")

# BrainTumour
print(f"{'*' * 20} BrainTumour {'*' * 20}")
bt_res["our"] = test_lowres(_dataset_config['Task01_BrainTumour']['best_models_paths'].get('ms'), "BrainTumour | Best Model (Multiscale)")



******************** HippoCampus ********************
MSUNet3D: MODE=inference
******************* TEST LowRes HippoCampus | Best Model (Multiscale) *******************


Scale 3: 100%|██████████| 52/52 [00:00<00:00, 72.12it/s]


Results per scale of lowres:
Scale 0) Mean Dice: 0.8692	 Std: 0.0318	 Mean Pr. Class Dice: ['0.8781', '0.8603']	 Std Pr. Class: ['0.0318', '0.0375']		Max Dice(hippocampus_227.nii.gz): 0.9190	 Min Dice(hippocampus_282.nii.gz): 0.7972
Scale 1) Mean Dice: 0.8624	 Std: 0.0373	 Mean Pr. Class Dice: ['0.8685', '0.8562']	 Std Pr. Class: ['0.0406', '0.0419']		Max Dice(hippocampus_088.nii.gz): 0.9220	 Min Dice(hippocampus_320.nii.gz): 0.7437
Scale 2) Mean Dice: 0.8348	 Std: 0.0634	 Mean Pr. Class Dice: ['0.8459', '0.8237']	 Std Pr. Class: ['0.0771', '0.0642']		Max Dice(hippocampus_094.nii.gz): 0.9412	 Min Dice(hippocampus_320.nii.gz): 0.6151
Scale 3) Mean Dice: 0.7848	 Std: 0.1342	 Mean Pr. Class Dice: ['0.7787', '0.7910']	 Std Pr. Class: ['0.1954', '0.1770']		Max Dice(hippocampus_023.nii.gz): 1.0000	 Min Dice(hippocampus_015.nii.gz): 0.3750
Mean Dice across all scales: 0.8378 	 Std: 0.0332
Mean Dice across all scales of classess: ['0.8428', '0.8328'] 	 Std: ['0.0388', '0.0280']
Latex: 0.8692 \

Scale 3: 100%|██████████| 97/97 [00:01<00:00, 93.37it/s] 

Results per scale of lowres:
Scale 0) Mean Dice: 0.6932	 Std: 0.1593	 Mean Pr. Class Dice: ['0.7776', '0.5855', '0.7166']	 Std Pr. Class: ['0.1282', '0.2302', '0.2632']		Max Dice(BRATS_396.nii.gz): 0.9408	 Min Dice(BRATS_024.nii.gz): 0.2176
Scale 1) Mean Dice: 0.6934	 Std: 0.1542	 Mean Pr. Class Dice: ['0.7875', '0.5870', '0.7057']	 Std Pr. Class: ['0.1201', '0.2312', '0.2657']		Max Dice(BRATS_396.nii.gz): 0.9275	 Min Dice(BRATS_380.nii.gz): 0.2538
Scale 2) Mean Dice: 0.6517	 Std: 0.1384	 Mean Pr. Class Dice: ['0.7599', '0.5241', '0.6711']	 Std Pr. Class: ['0.1201', '0.2468', '0.2484']		Max Dice(BRATS_396.nii.gz): 0.8852	 Min Dice(BRATS_380.nii.gz): 0.2185
Scale 3) Mean Dice: 0.5669	 Std: 0.1616	 Mean Pr. Class Dice: ['0.6682', '0.4627', '0.5699']	 Std Pr. Class: ['0.1464', '0.2634', '0.2828']		Max Dice(BRATS_175.nii.gz): 0.8947	 Min Dice(BRATS_380.nii.gz): 0.1689
Mean Dice across all scales: 0.6513 	 Std: 0.0516
Mean Dice across all scales of classess: ['0.7483', '0.5398', '0.6658'] 	

### DataSimulation -- BackBone

In [11]:

def test_backbone(model_dir_str, desc: str = "", depth: int = 3):
	if model_dir_str is None or model_dir_str == "":
		print("No model directory provided.")
		return
	if not model_dir_str.startswith("/home/si-hj/Desktop/medsegnet/"):
		model_dir_str = "/home/si-hj/Desktop/medsegnet/" + model_dir_str
	model_dir = Path(model_dir_str)
	model_path, cfg = model_params(model_dir)
	
	scales = list(range(depth+1))


	base_dir = Path(cfg.dataset.base_path)
	if not str(base_dir).endswith("_test1"):
		base_dir = Path(f"{base_dir}_test1")

	pad_dir = base_dir / "fullres" / "pad"
	upsampling_dir = base_dir / "fullres" / "upsampled"
	datasets = {}

	def get_dataset(base_name_dir, name):
		datasets[name] = {}
		for scale in scales:
			img_dir = os.path.join(base_name_dir, f"scale{scale}", "imagesTs")
			lbl_dir = os.path.join(base_dir, "lowres", "downsampled", f"scale{scale}", "labelsTs")
			img_dir_sort = sorted(os.listdir(img_dir))
			lbl_dir_sort = sorted(os.listdir(lbl_dir))
			if (cfg.dataset.name == "Task01_BrainTumour"):
				dataset = ModalitiesDataset(cfg, "test", img_dir_sort, lbl_dir_sort, str(img_dir), str(lbl_dir))
			else:
				dataset = MedicalDecathlonDataset(cfg, "test", img_dir_sort, lbl_dir_sort, str(img_dir), str(lbl_dir))
			datasets[name][scale] = dataset

	get_dataset(pad_dir, "pad")
	get_dataset(upsampling_dir, "upsampled")

	model = instantiate(cfg.architecture.path, cfg, mode="inference")

	checkpoint = torch.load(model_path, map_location=device)
	model_state_dict = checkpoint["model_state_dict"]
	model.load_state_dict(model_state_dict)
	model.to(device)
	model.eval()

	# dsc_results_per_scale = {scale: [] for scale in scales} moved this inside so its not averaged over pad and upsampled....
	print(f"******************* TEST Pad/upsamp {desc + ' ' if desc != '' else ''}*******************")
	def test(dataset, name):
		dsc_results_per_scale = {scale: [] for scale in scales} 
		for scale in scales:
			with torch.no_grad():
				for idx, (image, label) in tqdm(enumerate(dataset[name][scale]), desc=f"Scale {scale}", total=len(dataset[name][scale])):
					
					x = image.unsqueeze(0).float().to(device)

					output = model.forward(x)

					output = output.argmax(dim=1).unsqueeze(0) 

					target_shape = tuple(label.shape)

					if name == "pad": # crop to target shape
						cp = tio.CropOrPad(target_shape, padding_mode="constant")
						tio_image = tio.ScalarImage(tensor=output.squeeze(0))

						scaled_output = cp(tio_image)
						scaled_output = (scaled_output.data).unsqueeze(0)
					else: # downsample to target shape
						scaled_output = F.interpolate(
							output.float(), size=target_shape, mode="nearest"
						)

					label = label.unsqueeze(0)

					pred = scaled_output.squeeze(0)
					# pred = torch.argmax(scaled_output, dim=1)

					dsc_pr_class = consistent_dice(pred.to(device), label.to(device), cfg.dataset.num_classes)
					dsc_results_per_scale[scale].append([dsc_class.item() for dsc_class in dsc_pr_class])

					
					
		print(f"Results per scale of {name}:")
		print_results(dsc_results_per_scale, dataset[name], latex=True)
		return dsc_results_per_scale
		
	pad_res = test(datasets, "pad")
	up_res = test(datasets, "upsampled")
	print("\n")
	return pad_res, up_res


# Hippocampus
print(f"{'*' * 20} HippoCampus {'*' * 20}")
hp_res["bb_pad"], hp_res["bb_up"] = test_backbone(_dataset_config['Task04_Hippocampus']['best_models_paths'].get('bb'), "HippoCampus | Best U-Net3D")
hp_res["bb-aug_pad"], hp_res["bb-aug_up"] = test_backbone(_dataset_config['Task04_Hippocampus']['best_models_paths'].get('aug'), "HippoCampus | Best U-Net-aug3D")

# Prostate
print(f"{'*' * 20} Prostate {'*' * 20}")
# test_backbone(_dataset_config['Task05_Prostate']['best_models_paths'].get('bb'), "Prostate | Best U-Net3D")
# test_backbone(_dataset_config['Task05_Prostate']['best_models_paths'].get('aug'), "Prostate | Best U-Net-aug3D")

# Braintumour
print(f"{'*' * 20} BrainTumour {'*' * 20}")
bt_res["bb_pad"], bt_res["bb_up"] = test_backbone(_dataset_config['Task01_BrainTumour']['best_models_paths'].get('bb'), "BrainTumour | Best U-Net3D")
bt_res["bb-aug_pad"], bt_res["bb-aug_up"] = test_backbone(_dataset_config['Task01_BrainTumour']['best_models_paths'].get('aug'), "BrainTumour | Best U-Net-aug3D")

******************** HippoCampus ********************
******************* TEST Pad/upsamp HippoCampus | Best U-Net3D *******************


Scale 3:   0%|          | 0/52 [00:00<?, ?it/s]/home/si-hj/.conda/envs/bp/lib/python3.10/site-packages/torchio/transforms/preprocessing/intensity/rescale.py:89: RuntimeWarning: Rescaling image "image" not possible because all the intensity values are the same
  image.set_data(self.rescale(image.data, mask, image_name))
Scale 3: 100%|██████████| 52/52 [00:03<00:00, 15.85it/s]


Results per scale of pad:
Scale 0) Mean Dice: 0.8741	 Std: 0.0369	 Mean Pr. Class Dice: ['0.8864', '0.8617']	 Std Pr. Class: ['0.0308', '0.0500']		Max Dice(hippocampus_227.nii.gz): 0.9347	 Min Dice(hippocampus_173.nii.gz): 0.7675
Scale 1) Mean Dice: 0.0343	 Std: 0.0332	 Mean Pr. Class Dice: ['0.0126', '0.0559']	 Std Pr. Class: ['0.0358', '0.0577']		Max Dice(hippocampus_376.nii.gz): 0.1270	 Min Dice(hippocampus_015.nii.gz): 0.0000
Scale 2) Mean Dice: 0.0000	 Std: 0.0000	 Mean Pr. Class Dice: ['0.0000', '0.0000']	 Std Pr. Class: ['0.0000', '0.0000']		Max Dice(hippocampus_003.nii.gz): 0.0000	 Min Dice(hippocampus_003.nii.gz): 0.0000
Scale 3) Mean Dice: 0.0000	 Std: 0.0000	 Mean Pr. Class Dice: ['0.0000', '0.0000']	 Std Pr. Class: ['0.0000', '0.0000']		Max Dice(hippocampus_003.nii.gz): 0.0000	 Min Dice(hippocampus_003.nii.gz): 0.0000
Mean Dice across all scales: 0.2271 	 Std: 0.3738
Mean Dice across all scales of classess: ['0.2247', '0.2294'] 	 Std: ['0.3820', '0.3658']
Latex: 0.8741 \pm 

Scale 3: 100%|██████████| 52/52 [00:02<00:00, 24.03it/s]


Results per scale of upsampled:
Scale 0) Mean Dice: 0.8741	 Std: 0.0369	 Mean Pr. Class Dice: ['0.8864', '0.8617']	 Std Pr. Class: ['0.0308', '0.0500']		Max Dice(hippocampus_227.nii.gz): 0.9347	 Min Dice(hippocampus_173.nii.gz): 0.7675
Scale 1) Mean Dice: 0.8476	 Std: 0.0577	 Mean Pr. Class Dice: ['0.8638', '0.8314']	 Std Pr. Class: ['0.0460', '0.0788']		Max Dice(hippocampus_242.nii.gz): 0.9195	 Min Dice(hippocampus_199.nii.gz): 0.6436
Scale 2) Mean Dice: 0.7764	 Std: 0.0830	 Mean Pr. Class Dice: ['0.7877', '0.7652']	 Std Pr. Class: ['0.0711', '0.1184']		Max Dice(hippocampus_242.nii.gz): 0.9028	 Min Dice(hippocampus_173.nii.gz): 0.4872
Scale 3) Mean Dice: 0.3339	 Std: 0.1502	 Mean Pr. Class Dice: ['0.5355', '0.1322']	 Std Pr. Class: ['0.2326', '0.1981']		Max Dice(hippocampus_286.nii.gz): 0.7500	 Min Dice(hippocampus_011.nii.gz): 0.0000
Mean Dice across all scales: 0.7080 	 Std: 0.2189
Mean Dice across all scales of classess: ['0.7684', '0.6477'] 	 Std: ['0.1393', '0.2996']
Latex: 0.874

...

Results per scale of upsampled:
Scale 0) Mean Dice: 0.6992	 Std: 0.1540	 Mean Pr. Class Dice: ['0.7885', '0.5808', '0.7285']	 Std Pr. Class: ['0.1206', '0.2356', '0.2559']		Max Dice(BRATS_396.nii.gz): 0.9391	 Min Dice(BRATS_024.nii.gz): 0.2581
Scale 1) Mean Dice: 0.6773	 Std: 0.1513	 Mean Pr. Class Dice: ['0.7708', '0.5474', '0.7137']	 Std Pr. Class: ['0.1235', '0.2428', '0.2490']		Max Dice(BRATS_463.nii.gz): 0.9208	 Min Dice(BRATS_024.nii.gz): 0.2418
Scale 2) Mean Dice: 0.6202	 Std: 0.1332	 Mean Pr. Class Dice: ['0.7321', '0.4658', '0.6628']	 Std Pr. Class: ['0.1270', '0.2440', '0.2416']		Max Dice(BRATS_463.nii.gz): 0.8517	 Min Dice(BRATS_302.nii.gz): 0.1505
Scale 3) Mean Dice: 0.4929	 Std: 0.1338	 Mean Pr. Class Dice: ['0.6186', '0.3666', '0.4936']	 Std Pr. Class: ['0.1457', '0.2641', '0.2860']		Max Dice(BRATS_115.nii.gz): 0.8442	 Min Dice(BRATS_302.nii.gz): 0.1307
Mean Dice across all scales: 0.6224 	 Std: 0.0801
Mean Dice across all scales of classess: ['0.7275', '0.4901', '0.6497'

Scale 3: 100%|██████████| 97/97 [02:16<00:00,  1.40s/it]


Results per scale of pad:
Scale 0) Mean Dice: 0.6881	 Std: 0.1437	 Mean Pr. Class Dice: ['0.7730', '0.5661', '0.7250']	 Std Pr. Class: ['0.1262', '0.2220', '0.2477']		Max Dice(BRATS_396.nii.gz): 0.9123	 Min Dice(BRATS_380.nii.gz): 0.2664
Scale 1) Mean Dice: 0.0724	 Std: 0.0576	 Mean Pr. Class Dice: ['0.0707', '0.0577', '0.0888']	 Std Pr. Class: ['0.0542', '0.1074', '0.1246']		Max Dice(BRATS_024.nii.gz): 0.3333	 Min Dice(BRATS_017.nii.gz): 0.0000
Scale 2) Mean Dice: 0.0325	 Std: 0.0383	 Mean Pr. Class Dice: ['0.0063', '0.0396', '0.0518']	 Std Pr. Class: ['0.0109', '0.0817', '0.0704']		Max Dice(BRATS_475.nii.gz): 0.1616	 Min Dice(BRATS_017.nii.gz): 0.0000
Scale 3) Mean Dice: 0.0457	 Std: 0.0936	 Mean Pr. Class Dice: ['0.0022', '0.0354', '0.0995']	 Std Pr. Class: ['0.0124', '0.1743', '0.2308']		Max Dice(BRATS_024.nii.gz): 0.3636	 Min Dice(BRATS_016.nii.gz): 0.0000
Mean Dice across all scales: 0.2097 	 Std: 0.2766
Mean Dice across all scales of classess: ['0.2130', '0.1747', '0.2413'] 	 St

Scale 3: 100%|██████████| 97/97 [02:35<00:00,  1.60s/it]

Results per scale of upsampled:
Scale 0) Mean Dice: 0.6881	 Std: 0.1437	 Mean Pr. Class Dice: ['0.7730', '0.5661', '0.7250']	 Std Pr. Class: ['0.1262', '0.2220', '0.2477']		Max Dice(BRATS_396.nii.gz): 0.9123	 Min Dice(BRATS_380.nii.gz): 0.2664
Scale 1) Mean Dice: 0.6734	 Std: 0.1390	 Mean Pr. Class Dice: ['0.7643', '0.5364', '0.7196']	 Std Pr. Class: ['0.1267', '0.2357', '0.2345']		Max Dice(BRATS_396.nii.gz): 0.9008	 Min Dice(BRATS_380.nii.gz): 0.2554
Scale 2) Mean Dice: 0.6268	 Std: 0.1184	 Mean Pr. Class Dice: ['0.7364', '0.4645', '0.6795']	 Std Pr. Class: ['0.1277', '0.2355', '0.2188']		Max Dice(BRATS_396.nii.gz): 0.8351	 Min Dice(BRATS_380.nii.gz): 0.2037
Scale 3) Mean Dice: 0.5089	 Std: 0.1191	 Mean Pr. Class Dice: ['0.6372', '0.3763', '0.5131']	 Std Pr. Class: ['0.1395', '0.2576', '0.2669']		Max Dice(BRATS_115.nii.gz): 0.8394	 Min Dice(BRATS_374.nii.gz): 0.2891
Mean Dice across all scales: 0.6243 	 Std: 0.0704
Mean Dice across all scales of classess: ['0.7277', '0.4858', '0.6593'

### DataSimulation -- NNUNet

In [30]:
from pprint import pp
from sympy import factor
import yaml


def test_nnUNet(pred_dir_str, desc: str = "", depth: int = 3):
	scales = list(range(depth+1))

	dataset_short_name, dataset_type = pred_dir_str.split("/")

	dsc_results_per_scale = {scale: [] for scale in scales}
	print(f"TEST LowRes{' ' + desc if desc != '' else ''}:")
	dataset = {
		"hp": "Task04_Hippocampus",
		"bt": "Task01_BrainTumour",
		"p": "Task05_Prostate",
	}
	dataset_name = dataset[dataset_short_name]

	for scale in scales:
		label_name_str = f"/home/si-hj/Desktop/datasets/{dataset_name}_test1/lowres/downsampled/scale{scale}/labelsTs" 
		label_name_dir = Path(label_name_str)

		pred_name_str = f"/home/si-hj/Desktop/nn-unet/predictions/{pred_dir_str}/scale{scale}"
		pred_name_dir = Path(pred_name_str)
		
		image_files = sorted([f for f in os.listdir(pred_name_dir) if f.endswith(".nii.gz")])
		label_files = sorted([f for f in os.listdir(label_name_dir) if f.endswith(".nii.gz")])




		for img_file, label_file in tqdm(zip(image_files, label_files), desc=f"Scale {scale}", total=len(image_files)):
			img_path = pred_name_dir / img_file
			label_path = label_name_dir / label_file
			

			image = nib.load(str(img_path)).get_fdata() # (W, H, D)
			scaled_label = nib.load(str(label_path)).get_fdata() # (W, H, D)

			factor = 1 / (2 ** scale)
			if dataset_type == "pad":
				# target shape
				target_shape = tuple(round(dim * factor) for dim in image.shape)

				tio_image = tio.ScalarImage(str(img_path)) # (C, W, H, D)
				cp = tio.CropOrPad(target_shape)
				tio_image = cp(tio_image)
				scaled_image = torch.from_numpy(tio_image.data.numpy()).squeeze(0) # (C, W, H, D) -> (C, D, H, W)

			elif dataset_type == "upsampled":
				tensor = torch.from_numpy(image).unsqueeze(0).unsqueeze(0) # (W, H, D) -> (1, 1, D, H, W)
				tensor_out = F.interpolate(tensor, scale_factor=factor, mode="nearest")
			
				scaled_image = tensor_out.squeeze(0).squeeze(0)
			else: 
				raise ValueError(f"Unknown dataset type: {dataset_type} (available: pad, upsampled)")
			
			scaled_label = torch.from_numpy(scaled_label)

			# get medsegnet/config/dataset/Task01_BrainTumour.yaml -> num_classes
			num_classes = yaml.safe_load(open(f"/home/si-hj/Desktop/medsegnet/config/dataset/{dataset_name}.yaml"))['num_classes']
			
			dsc_pr_class = consistent_dice(
				scaled_image.to(device), scaled_label.to(device), num_classes
			)
			dsc_results_per_scale[scale].append([dsc_class.item() for dsc_class in dsc_pr_class])
			
	print(f"Results per scale of lowres:")
	print_results(dsc_results_per_scale, latex=True)
	return dsc_results_per_scale
			


# Hippocampus
print(f"{'*' * 20} HippocCampus {'*' * 20}")
hp_res["nn_pad"] = test_nnUNet(f"hp/pad", f"HP - Pad Best achieved for nnU-net")
hp_res["nn_up"] = test_nnUNet(f"hp/upsampled", f"HP - Upsampled Best achieved for nnU-net")

# Prostate
# print(f"{'*' * 20} Prostate {'*' * 20}")
# test_nnUNet(f"p/pad", f"P - Pad | Best achieved for nnU-net")
# test_nnUNet(f"p/upsampled", f"P - Upsampled | Best achieved for nnU-net")


# Braintumour
print(f"{'*' * 20} BrainTumour {'*' * 20}")
bt_res["nn_pad"] = test_nnUNet(f"bt/pad", f"BT - Pad | Best achieved for nnU-net")
bt_res["nn_up"] = test_nnUNet(f"bt/upsampled", f"BT - Upsampled | Best achieved for nnU-net")

******************** HippocCampus ********************
TEST LowRes HP - Pad Best achieved for nnU-net:


Scale 0:   0%|          | 0/52 [00:00<?, ?it/s]

Scale 3: 100%|██████████| 52/52 [00:01<00:00, 35.89it/s]


Results per scale of lowres:
Scale 0) Mean Dice: 0.8797	 Std: 0.0321	 Mean Pr. Class Dice: ['0.8887', '0.8706']	 Std Pr. Class: ['0.0329', '0.0344']
Scale 1) Mean Dice: 0.8195	 Std: 0.0448	 Mean Pr. Class Dice: ['0.8346', '0.8043']	 Std Pr. Class: ['0.0384', '0.0628']
Scale 2) Mean Dice: 0.5285	 Std: 0.0966	 Mean Pr. Class Dice: ['0.5517', '0.5053']	 Std Pr. Class: ['0.1107', '0.1128']
Scale 3) Mean Dice: 0.0297	 Std: 0.0619	 Mean Pr. Class Dice: ['0.0539', '0.0055']	 Std Pr. Class: ['0.1200', '0.0392']
Mean Dice across all scales: 0.5643 	 Std: 0.3360
Mean Dice across all scales of classess: ['0.5822', '0.5465'] 	 Std: ['0.3308', '0.3413']
Latex: 0.8797 \pm 0.0321 & 0.8195 \pm 0.0448 & 0.5285 \pm 0.0966 & 0.0297 \pm 0.0619 & 0.5643 \pm 0.3360 \\

TEST LowRes HP - Upsampled Best achieved for nnU-net:


Scale 3: 100%|██████████| 52/52 [00:00<00:00, 332.32it/s]


Results per scale of lowres:
Scale 0) Mean Dice: 0.8797	 Std: 0.0321	 Mean Pr. Class Dice: ['0.8887', '0.8707']	 Std Pr. Class: ['0.0329', '0.0344']
Scale 1) Mean Dice: 0.8197	 Std: 0.0666	 Mean Pr. Class Dice: ['0.8315', '0.8079']	 Std Pr. Class: ['0.0665', '0.0751']
Scale 2) Mean Dice: 0.5972	 Std: 0.1062	 Mean Pr. Class Dice: ['0.5857', '0.6087']	 Std Pr. Class: ['0.1348', '0.1212']
Scale 3) Mean Dice: 0.0551	 Std: 0.0847	 Mean Pr. Class Dice: ['0.0872', '0.0230']	 Std Pr. Class: ['0.1348', '0.0815']
Mean Dice across all scales: 0.5879 	 Std: 0.3251
Mean Dice across all scales of classess: ['0.5983', '0.5776'] 	 Std: ['0.3163', '0.3345']
Latex: 0.8797 \pm 0.0321 & 0.8197 \pm 0.0666 & 0.5972 \pm 0.1062 & 0.0551 \pm 0.0847 & 0.5879 \pm 0.3251 \\

******************** BrainTumour ********************
TEST LowRes BT - Pad | Best achieved for nnU-net:


Scale 3: 100%|██████████| 97/97 [00:08<00:00, 11.67it/s]


Results per scale of lowres:
Scale 0) Mean Dice: 0.7124	 Std: 0.1518	 Mean Pr. Class Dice: ['0.7942', '0.5913', '0.7517']	 Std Pr. Class: ['0.1117', '0.2531', '0.2433']
Scale 1) Mean Dice: 0.6696	 Std: 0.1524	 Mean Pr. Class Dice: ['0.7653', '0.5375', '0.7060']	 Std Pr. Class: ['0.1201', '0.2559', '0.2477']
Scale 2) Mean Dice: 0.5828	 Std: 0.1278	 Mean Pr. Class Dice: ['0.6818', '0.4682', '0.5984']	 Std Pr. Class: ['0.1332', '0.2340', '0.2295']
Scale 3) Mean Dice: 0.3990	 Std: 0.1253	 Mean Pr. Class Dice: ['0.4905', '0.3298', '0.3766']	 Std Pr. Class: ['0.1690', '0.2107', '0.2372']
Mean Dice across all scales: 0.5909 	 Std: 0.1203
Mean Dice across all scales of classess: ['0.6830', '0.4817', '0.6082'] 	 Std: ['0.1185', '0.0980', '0.1448']
Latex: 0.7124 \pm 0.1518 & 0.6696 \pm 0.1524 & 0.5828 \pm 0.1278 & 0.3990 \pm 0.1253 & 0.5909 \pm 0.1203 \\

TEST LowRes BT - Upsampled | Best achieved for nnU-net:


Scale 3: 100%|██████████| 97/97 [00:03<00:00, 28.26it/s]

Results per scale of lowres:
Scale 0) Mean Dice: 0.7124	 Std: 0.1518	 Mean Pr. Class Dice: ['0.7942', '0.5913', '0.7517']	 Std Pr. Class: ['0.1117', '0.2531', '0.2433']
Scale 1) Mean Dice: 0.6597	 Std: 0.1551	 Mean Pr. Class Dice: ['0.7500', '0.5184', '0.7109']	 Std Pr. Class: ['0.1266', '0.2546', '0.2496']
Scale 2) Mean Dice: 0.5954	 Std: 0.1403	 Mean Pr. Class Dice: ['0.6984', '0.4442', '0.6435']	 Std Pr. Class: ['0.1374', '0.2444', '0.2533']
Scale 3) Mean Dice: 0.4524	 Std: 0.1447	 Mean Pr. Class Dice: ['0.5535', '0.3214', '0.4822']	 Std Pr. Class: ['0.1854', '0.2329', '0.2877']
Mean Dice across all scales: 0.6050 	 Std: 0.0974
Mean Dice across all scales of classess: ['0.6990', '0.4688', '0.6471'] 	 Std: ['0.0906', '0.0997', '0.1027']
Latex: 0.7124 \pm 0.1518 & 0.6597 \pm 0.1551 & 0.5954 \pm 0.1403 & 0.4524 \pm 0.1447 & 0.6050 \pm 0.0974 \\



# Temporary to print tables

This one is for overall mean

In [46]:
import numpy as np

def generate_latex_table(results_dict, model_names_map, table_title, scales_order=[0, 1, 2, 3]):
    """
    Generates and prints LaTeX table rows from a dictionary of results.
    It highlights the best (bold) and second-best (underline) values in each column
    by comparing values rounded to 3 decimal places, correctly handling all ties.

    Args:
        results_dict (dict): The dictionary containing the model results (e.g., hp_res, bt_res).
        model_names_map (dict): A map from keys in results_dict to display names for the table.
        table_title (str): The title to print above the table.
        scales_order (list): The order of scales to process.
    """
    print(f"\n\n{'='*20} LaTeX Table Rows for {table_title} {'='*20}")
    
    # --- PASS 1: Pre-compute all data ---
    all_models_data = []
    model_keys_in_order = [key for key in model_names_map.keys() if key in results_dict and results_dict[key] is not None]

    for model_key in model_keys_in_order:
        dsc_results_per_scale = results_dict[model_key]
        if not all(s in dsc_results_per_scale for s in scales_order):
            continue
        
        model_means = []
        model_stds = []
        for scale in scales_order:
            results_list = dsc_results_per_scale[scale]
            results_mean_per_image = np.mean(results_list, axis=1)
            model_means.append(np.mean(results_mean_per_image))
            model_stds.append(np.std(results_mean_per_image))

        overall_mean = np.mean(model_means)
        overall_std = np.std(model_means)
        model_means.append(overall_mean)
        model_stds.append(overall_std)
        
        all_models_data.append({
            'key': model_key,
            'name': model_names_map[model_key],
            'means': model_means,
            'stds': model_stds
        })

    if not all_models_data:
        print(f"No valid data found for table: {table_title}")
        return

    # --- PASS 2: Analyze columns using rounded values and generate table ---
    means_array = np.array([data['means'] for data in all_models_data])
    num_cols = means_array.shape[1]
    
    max_vals_per_col = []
    second_max_vals_per_col = []

    for j in range(num_cols):
        # --- ROBUST LOGIC USING ROUNDING ---
        # 1. Round all values in the column to 3 decimal places for comparison
        col_values_rounded = np.round(means_array[:, j], 3)
        
        # 2. Get the unique sorted values from the rounded data
        unique_sorted_desc = np.unique(col_values_rounded)[::-1]
        
        # 3. Assign the top two unique rounded values
        max_val = unique_sorted_desc[0] if len(unique_sorted_desc) > 0 else -1.0
        second_max_val = unique_sorted_desc[1] if len(unique_sorted_desc) > 1 else -1.0
        
        max_vals_per_col.append(max_val)
        second_max_vals_per_col.append(second_max_val)

    # --- Print Data Rows ---
    for i, model_data in enumerate(all_models_data):
        latex_parts = []
        for j in range(num_cols):
            mean = model_data['means'][j]
            std = model_data['stds'][j]
            
            # Use the original mean for display formatting, but the rounded mean for logic
            mean_for_comparison = round(mean, 3)
            mean_str_for_display = f"{mean:.3f}"
            
            max_val = max_vals_per_col[j]
            second_max_val = second_max_vals_per_col[j]

            # Compare the rounded value against the stored rounded max/second_max
            if mean_for_comparison == max_val:
                formatted_mean = f"\\textbf{{{mean_str_for_display}}}"
            elif mean_for_comparison == second_max_val:
                formatted_mean = f"\\underline{{{mean_str_for_display}}}"
            else:
                formatted_mean = mean_str_for_display
            
            latex_parts.append(f"{formatted_mean}$\\pm${std:.3f}")
        
        row_content = " & ".join(latex_parts)
        
        model_key = model_data['key']
        
        if model_key.endswith('_up') and i < len(all_models_data) - 1:
             print(f"{model_data['name']} & {row_content} \\\\\\midrule")
        else:
             print(f"{model_data['name']} & {row_content} \\\\")

    print(f"{'='*20} End of Table {'='*20}\n")


# The rest of your script remains the same
model_names = {
    "bb_pad": "UNet--Pad",
    "bb_up": "UNet--Up",
    "bb-aug_pad": "UNet+Aug--Pad",
    "bb-aug_up": "UNet+Aug--Up",
    "nn_pad": "nnUNet--Pad",
    "nn_up": "nnUNet--Up",
    "our": "RARE-UNet",
}

# The rest of your script, including where you call this function, remains the same.
generate_latex_table(hp_res, model_names, "Hippocampus")
generate_latex_table(bt_res, model_names, "BrainTumour")



==================== LaTeX Table Rows for Hippocampus ====================
UNet--Pad & \underline{0.874}$\pm$0.037 & 0.034$\pm$0.033 & 0.000$\pm$0.000 & 0.000$\pm$0.000 & 0.227$\pm$0.374 \\
UNet--Up & \underline{0.874}$\pm$0.037 & 0.848$\pm$0.058 & 0.776$\pm$0.083 & 0.334$\pm$0.150 & 0.708$\pm$0.219 \\\midrule
UNet+Aug--Pad & 0.871$\pm$0.034 & 0.213$\pm$0.034 & 0.066$\pm$0.026 & 0.000$\pm$0.000 & 0.287$\pm$0.345 \\
UNet+Aug--Up & 0.871$\pm$0.034 & \underline{0.860}$\pm$0.039 & \underline{0.822}$\pm$0.069 & \textbf{0.789}$\pm$0.131 & \underline{0.835}$\pm$0.032 \\\midrule
nnUNet--Pad & \textbf{0.880}$\pm$0.032 & 0.819$\pm$0.045 & 0.529$\pm$0.097 & 0.030$\pm$0.062 & 0.564$\pm$0.336 \\
nnUNet--Up & \textbf{0.880}$\pm$0.032 & 0.820$\pm$0.067 & 0.597$\pm$0.106 & 0.055$\pm$0.085 & 0.588$\pm$0.325 \\\midrule
RARE-UNet & 0.869$\pm$0.032 & \textbf{0.862}$\pm$0.037 & \textbf{0.835}$\pm$0.063 & \underline{0.785}$\pm$0.134 & \textbf{0.838}$\pm$0.033 \\
==================== End of Table =========

This one is for all the classes

In [50]:
import numpy as np

def generate_class_table_grouped(hp_res, bt_res):
    """
    Generates a LaTeX table for per-class Dice scores, applying formatting
    (bold for best, underline for second-best) INDEPENDENTLY within each scale group.
    """
    
    # --- Configuration: Define the structure of the table ---
    columns = [
        ('HP--L', hp_res, 0), ('HP--R', hp_res, 1),
        ('BT--PE', bt_res, 0), ('BT--NC', bt_res, 1), ('BT--ET', bt_res, 2),
    ]
    model_definitions = [
        ("UNet--", "bb_pad", "bb_up"),
        ("UNet+Aug--", "bb-aug_pad", "bb-aug_up"),
        ("nnUNet--", "nn_pad", "nn_up"),
    ]
    rare_unet_def = ("RARE-UNet", "our")
    scales_config = [
        (0, "1"), (1, "1/2"), (2, "1/4"), (3, "1/8"), ('overall', "Overall")
    ]

    # --- PASS 1: Collect all data, tagging each row with its scale group ---
    all_rows_data = []
    for scale, scale_name in scales_config:
        # Define the set of rows for the current scale group
        group_rows = []
        if scale == 0:
            for model_name, pad_key, _ in model_definitions:
                group_rows.append({'label': f"{model_name} ({scale_name})", 'model_key': pad_key})
        elif scale != 'overall':
            for model_name, pad_key, up_key in model_definitions:
                group_rows.append({'label': f"{model_name} Pad ({scale_name})", 'model_key': pad_key})
                group_rows.append({'label': f"{model_name} Up ({scale_name})", 'model_key': up_key})
        
        # Add RARE-UNet to all non-overall groups
        if scale != 'overall':
            group_rows.append({'label': f"{rare_unet_def[0]} ({scale_name})", 'model_key': rare_unet_def[1]})
        
        # Define rows for the 'Overall' group separately
        if scale == 'overall':
             for model_name, pad_key, up_key in model_definitions:
                group_rows.append({'label': f"{model_name} Pad ({scale_name})", 'model_key': pad_key})
                group_rows.append({'label': f"{model_name} Up ({scale_name})", 'model_key': up_key})
             group_rows.append({'label': f"{rare_unet_def[0]} ({scale_name})", 'model_key': rare_unet_def[1]})
        
        # Calculate and store the data for each defined row
        for row_info in group_rows:
            row_data = {'label': row_info['label'], 'scale_group': scale_name, 'means': [], 'stds': []}
            model_key = row_info['model_key']
            
            for _, res_dict, class_idx in columns:
                if scale != 'overall':
                    class_scores = np.array(res_dict[model_key][scale])[:, class_idx]
                    mean, std = np.mean(class_scores), np.std(class_scores)
                else: # Handle 'Overall' calculation for this specific row
                    means_per_scale = []
                    # Overall for Pad models averages scales 0,1,2,3 from the pad_key
                    if 'Pad' in row_info['label'] or 'UNet (1' in row_info['label']:
                        key_to_use = model_definitions[[m[0] for m in model_definitions].index(row_info['label'].split(' ')[0])][1]
                        for s in range(4):
                            means_per_scale.append(np.mean(np.array(res_dict[key_to_use][s])[:, class_idx]))
                    # Overall for Up models averages scales 1,2,3 from the up_key (as there's no scale 0 for 'Up')
                    elif 'Up' in row_info['label']:
                         key_to_use = model_definitions[[m[0] for m in model_definitions].index(row_info['label'].split(' ')[0])][2]
                         for s in range(1, 4):
                            means_per_scale.append(np.mean(np.array(res_dict[key_to_use][s])[:, class_idx]))
                    # Overall for RARE-UNet
                    else:
                        for s in range(4):
                           means_per_scale.append(np.mean(np.array(res_dict[model_key][s])[:, class_idx]))

                    mean, std = np.mean(means_per_scale), np.std(means_per_scale)
                
                row_data['means'].append(mean)
                row_data['stds'].append(std)
            all_rows_data.append(row_data)

    # --- PASS 2: Analyze and print, group by group ---
    print("\n\n" + "="*20 + " LaTeX Table for Per-Class DSC (Grouped) " + "="*20)
    
    for idx, (scale, scale_name) in enumerate(scales_config):
        # 1. Filter data for the current group
        group_rows = [row for row in all_rows_data if row['scale_group'] == scale_name]
        if not group_rows: continue

        # 2. Analyze this group's columns to find best/second-best values
        group_means_array = np.array([data['means'] for data in group_rows])
        max_vals_for_group = []
        second_max_vals_for_group = []
        
        for j in range(group_means_array.shape[1]):
            col_values_rounded = np.round(group_means_array[:, j], 3)
            unique_sorted_desc = np.unique(col_values_rounded)[::-1]
            max_vals_for_group.append(unique_sorted_desc[0] if len(unique_sorted_desc) > 0 else -1.0)
            second_max_vals_for_group.append(unique_sorted_desc[1] if len(unique_sorted_desc) > 1 else -1.0)

        # 3. Print the rows for this group with the correct formatting
        for row_data in group_rows:
            latex_parts = []
            for j in range(len(columns)):
                mean, std = row_data['means'][j], row_data['stds'][j]
                mean_for_comparison = round(mean, 3)
                mean_str_for_display = f"{mean:.3f}"
                
                if mean_for_comparison == max_vals_for_group[j]:
                    formatted_mean = f"\\textbf{{{mean_str_for_display}}}"
                elif mean_for_comparison == second_max_vals_for_group[j]:
                    formatted_mean = f"\\underline{{{mean_str_for_display}}}"
                else:
                    formatted_mean = mean_str_for_display
                
                latex_parts.append(f"{formatted_mean}$\\pm${std:.3f}")
            
            print(f"{row_data['label']} & " + " & ".join(latex_parts) + " \\\\")

        # 4. Print \midrule between groups, but not after the last one
        if idx < len(scales_config) - 1:
            print("\\midrule")
            
    print("="*20 + " End of Table " + "="*20 + "\n")


# --- How to use ---
# At the end of your script, after `hp_res` and `bt_res` are populated, call:
generate_class_table_grouped(hp_res, bt_res)




==================== LaTeX Table for Per-Class DSC (Grouped) ====================
UNet-- (1) & \underline{0.886}$\pm$0.031 & \underline{0.862}$\pm$0.050 & \underline{0.788}$\pm$0.121 & 0.581$\pm$0.236 & \underline{0.728}$\pm$0.256 \\
UNet+Aug-- (1) & 0.879$\pm$0.033 & \underline{0.862}$\pm$0.038 & 0.773$\pm$0.126 & 0.566$\pm$0.222 & 0.725$\pm$0.248 \\
nnUNet-- (1) & \textbf{0.889}$\pm$0.033 & \textbf{0.871}$\pm$0.034 & \textbf{0.794}$\pm$0.112 & \textbf{0.591}$\pm$0.253 & \textbf{0.752}$\pm$0.243 \\
RARE-UNet (1) & 0.878$\pm$0.032 & 0.860$\pm$0.037 & 0.778$\pm$0.128 & \underline{0.586}$\pm$0.230 & 0.717$\pm$0.263 \\
\midrule
UNet-- Pad (1/2) & 0.013$\pm$0.036 & 0.056$\pm$0.058 & 0.332$\pm$0.226 & 0.253$\pm$0.246 & 0.041$\pm$0.050 \\
UNet-- Up (1/2) & 0.864$\pm$0.046 & 0.831$\pm$0.079 & \underline{0.771}$\pm$0.124 & \underline{0.547}$\pm$0.243 & \underline{0.714}$\pm$0.249 \\
UNet+Aug-- Pad (1/2) & 0.258$\pm$0.050 & 0.168$\pm$0.040 & 0.071$\pm$0.054 & 0.058$\pm$0.107 & 0.089$\pm$0.125

# Reconstruction when time allows or bored

In [11]:
PROJECT_ROOT = Path('/home/si-hj/Desktop/medsegnet')
DATA_ROOT = Path('/home/si-hj/Desktop/datasets')
NNUNET_PRED_ROOT = Path('/home/si-hj/Desktop/nn-unet/predictions')
TRAINED_MODELS_ROOT = PROJECT_ROOT / "trained_models"

# Update sys.path
sys.path.append(str(PROJECT_ROOT.parent)) # Desktop`
sys.path.append(str(PROJECT_ROOT))


DATASET_CONFIGS = {
    "Task01_BrainTumour": {
        "num_classes": OmegaConf.load(PROJECT_ROOT / "config/dataset/Task01_BrainTumour.yaml").num_classes,
        "dataset_class": ModalitiesDataset,
        "base_path_segment": "Task01_BrainTumour_test1"
    },
    "Task04_Hippocampus": {
        "num_classes": OmegaConf.load(PROJECT_ROOT / "config/dataset/Task04_Hippocampus.yaml").num_classes,
        "dataset_class": MedicalDecathlonDataset,
        "base_path_segment": "Task04_Hippocampus_test1"
    },
}

# For nnU-Net (mapping short names to full names)
NNUNET_DATASET_MAP = {
    "hp": "Task04_Hippocampus",
    "bt": "Task01_BrainTumour",
}



def load_model_config(model_dir: Path) -> Tuple[Path, DictConfig, str]:
    if not model_dir.is_dir():
        raise FileNotFoundError(f"Model directory not found: {model_dir}")
    
    model_path = model_dir / "best_model.pth"
    cfg_path = model_dir / "config.yaml"
    if not model_path.exists() or not cfg_path.exists():
        raise FileNotFoundError(f"best_model.pth or config.yaml not found in {model_dir}")
        
    cfg = OmegaConf.load(cfg_path)
    setup_seed(cfg.seed)
    
    if not isinstance(cfg, DictConfig):
        raise TypeError("cfg must be a DictConfig.")
    
    return model_path, cfg, cfg.dataset.name


def get_dataset_for_scale(
    cfg: DictConfig,
    dataset_name_key: str, # e.g., "Task04_Hippocampus"
    scale: int,
    img_base_dir: Path,
    lbl_base_dir: Path, # This might vary for nnU-Net where labels are always from original lowres
    split: str = "test"
) -> torch.utils.data.Dataset:

    dataset_cfg = DATASET_CONFIGS[dataset_name_key]
    DatasetClass = dataset_cfg["dataset_class"]

    img_dir = img_base_dir / f"scale{scale}" / "imagesTs"
    lbl_dir = lbl_base_dir / f"scale{scale}" / "labelsTs" # Adjust if labels are always fullres/lowres

    if not img_dir.exists() or not lbl_dir.exists():
        raise FileNotFoundError(f"Image or label directory not found for scale {scale} under {img_base_dir} or {lbl_base_dir}")

    img_files_sorted = sorted(os.listdir(img_dir))
    lbl_files_sorted = sorted(os.listdir(lbl_dir))
    
    return DatasetClass(cfg, split, img_files_sorted, lbl_files_sorted, str(img_dir), str(lbl_dir))

# Example usage in test_lowres
# base_dataset_path = DATA_ROOT / DATASET_CONFIGS[dataset_name_from_cfg]["base_path_segment"]
# lowres_data_root = base_dataset_path / "lowres" / "downsampled"
# datasets[scale] = get_dataset_for_scale(cfg, dataset_name_from_cfg, scale, lowres_data_root, lowres_data_root)

In [12]:
def evaluate_model_on_scales(
    model: torch.nn.Module,
    cfg: DictConfig,
    datasets_per_scale: Dict[int, torch.utils.data.Dataset],
    device: torch.device,
    prediction_processor_fn: callable, # Takes model output, label shape, returns processed pred
    desc_prefix: str = ""
) -> Dict[int, list]:
    
    dsc_results = {scale: [] for scale in datasets_per_scale.keys()}
    model.eval()

    for scale, dataset in datasets_per_scale.items():
        dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=cfg.training.num_workers)
        with torch.no_grad():
            for idx, (image, label) in tqdm(enumerate(dataloader), total=len(dataloader), desc=f"{desc_prefix} Scale {scale}"):
                
                # Common part: move image to device
                image_dev = image.to(device) # Assuming image always needs .to(device)
                                            # For backbone, it's image.unsqueeze(0).float().to(device)
                                            # This needs standardization or to be part of prediction_processor_fn

                # Model-specific inference and prediction processing
                # This part would be handled by prediction_processor_fn
                # Example: output = model(image_dev) or model.run_inference(image_dev)
                #          scaled_output = post_process(output, label.shape)
                #          pred = torch.argmax(scaled_output, dim=1).squeeze(0)
                
                pred = prediction_processor_fn(model, image_dev, label.shape) # Pass label.shape for target
                
                label_dev = label.squeeze(0).to(device) # Assuming label always needs squeeze and .to(device)
                
                dsc_pr_class = consistent_dice(pred, label_dev, cfg) # cfg needs num_classes
                dsc_results[scale].append([dsc_class.item() for dsc_class in dsc_pr_class])
                
                # Optional: conditional plotting logic here or passed as another callback
    return dsc_results

In [13]:
def test_lowres(model_dir_str: str, desc: str = "", num_ms_levels: int = 3):
    model_dir = Path(model_dir_str)
    model_path, cfg, dataset_name_from_cfg = load_model_config(model_dir) # Updated

    scales = list(range(num_ms_levels + 1))

    dataset_cfg_details = DATASET_CONFIGS[dataset_name_from_cfg]
    base_data_path = DATA_ROOT / dataset_cfg_details["base_path_segment"]
    lowres_data_root = base_data_path / "lowres" / "downsampled"

    datasets = {}
    filenames_per_scale = {} # For print_results if needed

    for scale in scales:
        # Use the new get_dataset_for_scale
        dataset_instance = get_dataset_for_scale(cfg, dataset_name_from_cfg, scale, lowres_data_root, lowres_data_root)
        datasets[scale] = dataset_instance
        # If print_results needs filenames:
        # filenames_per_scale[scale] = [dataset_instance.img_paths[i] for i in range(len(dataset_instance))]


    model = instantiate(cfg.architecture.path, cfg, "inference")
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint["model_state_dict"])
    model.to(device)
    model.eval()

    dsc_results_per_scale = {scale: [] for scale in scales}

    print(f"******************* TEST LowRes {desc + ' ' if desc else ''}*******************")
    for scale in scales:
        dataloader = DataLoader(datasets[scale], batch_size=1, shuffle=False, num_workers=cfg.training.num_workers)
        with torch.no_grad():
            for idx, (image, label) in tqdm(enumerate(dataloader), total=len(dataloader), desc=f"Scale {scale}"):
                output = model.run_inference(image.to(device)) # Specific to MS model
                pred = torch.argmax(output, dim=1).squeeze(0)
                label_squeezed = label.squeeze(0) # Keep original label for plotting if needed
                
                dsc_pr_class = consistent_dice(pred, label_squeezed.to(device), cfg)
                dsc_results_per_scale[scale].append([dsc_class.item() for dsc_class in dsc_pr_class])

                # Conditional plotting (can be refactored too)
                # if plot and (scale == 0 and np.mean([d.item() for d in dsc_pr_class]) <= 0.3):
                #    plot_img(image, pred, label_squeezed)

    print(f"Results per scale of lowres:")
    print_results(dsc_results_per_scale, None) # Pass filenames if needed

In [14]:
results = {
    "Multi-Scale": {
        "LowRes": {
            0: {"dice": [0.0, 0.0], "std": [0.0, 0.0]},
            1: {"dice": [0.0, 0.0], "std": [0.0, 0.0]},
            2: {"dice": [0.0, 0.0], "std": [0.0, 0.0]},
            3: {"dice": [0.0, 0.0], "std": [0.0, 0.0]},
        }
    },
    "U-Net": {
        "Pad": {
            0: {"dice": [0.0, 0.0], "std": [0.0, 0.0]},
            1: {"dice": [0.0, 0.0], "std": [0.0, 0.0]},
            2: {"dice": [0.0, 0.0], "std": [0.0, 0.0]},
            3: {"dice": [0.0, 0.0], "std": [0.0, 0.0]},
        },
        "Upsample": {
            0: {"dice": [0.0, 0.0], "std": [0.0, 0.0]},
            1: {"dice": [0.0, 0.0], "std": [0.0, 0.0]},
            2: {"dice": [0.0, 0.0], "std": [0.0, 0.0]},
            3: {"dice": [0.0, 0.0], "std": [0.0, 0.0]},
        }
    },
    "U-Net+Aug": {
        "Pad": {
            0: {"dice": [0.0, 0.0], "std": [0.0, 0.0]},
            1: {"dice": [0.0, 0.0], "std": [0.0, 0.0]},
            2: {"dice": [0.0, 0.0], "std": [0.0, 0.0]},
            3: {"dice": [0.0, 0.0], "std": [0.0, 0.0]},
        },
        "Upsample": {
            0: {"dice": [0.0, 0.0], "std": [0.0, 0.0]},
            1: {"dice": [1.0, 0.0], "std": [1.0, 0.0]},
            2: {"dice": [0.0, 0.0], "std": [0.0, 0.0]},
            3: {"dice": [0.0, 0.0], "std": [0.0, 0.0]},
        }
    },
    "nnU-Net": {
        "Pad": {
            0: {"dice": [0.0, 0.0], "std": [0.0, 0.0]},
            1: {"dice": [0.0, 0.0], "std": [0.0, 0.0]},
            2: {"dice": [0.0, 0.0], "std": [0.0, 0.0]},
            3: {"dice": [0.0, 0.0], "std": [0.0, 0.0]},
        },
        "Upsample": {
            0: {"dice": [0.0, 0.0], "std": [0.0, 0.0]},
            1: {"dice": [1.0, 0.0], "std": [0.0, 0.0]},
            2: {"dice": [1.0, 0.0], "std": [0.0, 0.0]},
            3: {"dice": [0.0, 0.0], "std": [0.0, 0.0]},
        }
    }
}

In [15]:
# Generate the exact LaTeX table structure from `results` with zeros, matching the user’s template.

import numpy as np




def format_cell(mean, std, bold=False, underline=False):
    """Return cell text matching the user's math formatting."""
    if bold:
        return f"$\\mathbf{{{mean:.4f}}} \\pm {std:.4f}$"
    if underline:
        return f"\\underline{{{mean:.4f}}} $\\pm$ {std:.4f}"
    return f"{mean:.4f} $\\pm$ {std:.4f}"

# Flatten rows and compute per-scale mean/std using only the first class,
# average only for the final column
rows = []
for method, variants in results.items():
    for variant, scales in variants.items():
        # Use only the first class's metrics for wide table columns
        means = [scales[s]["dice"][0] for s in range(4)]
        stds  = [scales[s]["std"][0] for s in range(4)]
        avg_mean, avg_std = np.mean(means), np.mean(stds)
        rows.append({
            "method": method,
            "variant": variant,
            0:  (means[0], stds[0]),
            1:  (means[1], stds[1]),
            2:  (means[2], stds[2]),
            3:  (means[3], stds[3]),
            "avg": (avg_mean, avg_std)
        })

# Find top-2 for each column by mean
cols = [0, 1, 2, 3, "avg"]
top2 = {c: sorted(rows, key=lambda r: r[c][0], reverse=True)[:2] for c in cols}

# Build and print LaTeX
lines = []
a = lines.append
a(r"\begin{table}[H]")
a(r"  \centering")
a(r"  \footnotesize")
a(r"  \caption{Mean Dice Similarity Coefficient (DSC) across scales and model families from the \texttt{BrainTumour test} datasets.}")
a(r"  \label{tab:mean-dsc-bt}")
a(r"\begin{adjustbox}{max width=1.1\textwidth}")
a(r"    \hspace{-5em}")
a(r"    \begin{tabular}{@{}L{2cm} C{3cm} L{2cm} C{2.5cm} C{3cm} C{3cm} c@{}}")
a(r"      \toprule")
a(r"      \textbf{Method} & \textbf{DSC@res} & \textbf{Variant} & \textbf{DSC@res/2} & \textbf{DSC@res/4} & \textbf{DSC@res/8} & \textbf{Avg. across scales} \\")
a(r"      \midrule")

for method, variants in results.items():
    if len(variants) == 1:  # Multi-Scale
        var = next(iter(variants))
        ent = next(r for r in rows if r["method"]==method and r["variant"]==var)
        a(r"      \rowcolor{gray!15}")
        cells = []
        for c in cols:
            mean, std = ent[c]
            bold = ent in top2[c][:1]
            underline = ent in top2[c][1:2]
            cells.append(format_cell(mean, std, bold, underline))
        a(f"      {method} & {cells[0]} & {var} & {cells[1]} & {cells[2]} & {cells[3]} & {cells[4]} \\\\")
        a(r"      \midrule")
    else:
        n = len(variants)
        for i, var in enumerate(variants):
            ent = next(r for r in rows if r["method"]==method and r["variant"]==var)
            cells = []
            for c in cols:
                mean, std = ent[c]
                bold = ent in top2[c][:1]
                underline = ent in top2[c][1:2]
                cells.append(format_cell(mean, std, bold, underline))
            if i == 0:
                a(f"      \\multirow{{{n}}}{{*}}{{{method}}} & \\multirow{{{n}}}{{*}}{{{cells[0]}}} & {var} & {cells[1]} & {cells[2]} & {cells[3]} & {cells[4]} \\\\")
            else:
                a(r"        &")
                a(f"        & {var} & {cells[1]} & {cells[2]} & {cells[3]} & {cells[4]} \\\\")
        a(r"      \midrule")

# Replace last midrule with bottomrule
for i in range(len(lines)-1, -1, -1):
    if lines[i].strip() == r"\midrule":
        lines[i] = r"      \bottomrule"
        break

a(r"    \end{tabular}")
a(r"  \end{adjustbox}")
a(r"\end{table}")

print("\n".join(lines))



\begin{table}[H]
  \centering
  \footnotesize
  \caption{Mean Dice Similarity Coefficient (DSC) across scales and model families from the \texttt{BrainTumour test} datasets.}
  \label{tab:mean-dsc-bt}
\begin{adjustbox}{max width=1.1\textwidth}
    \hspace{-5em}
    \begin{tabular}{@{}L{2cm} C{3cm} L{2cm} C{2.5cm} C{3cm} C{3cm} c@{}}
      \toprule
      \textbf{Method} & \textbf{DSC@res} & \textbf{Variant} & \textbf{DSC@res/2} & \textbf{DSC@res/4} & \textbf{DSC@res/8} & \textbf{Avg. across scales} \\
      \midrule
      \rowcolor{gray!15}
      Multi-Scale & $\mathbf{0.0000} \pm 0.0000$ & LowRes & 0.0000 $\pm$ 0.0000 & \underline{0.0000} $\pm$ 0.0000 & $\mathbf{0.0000} \pm 0.0000$ & 0.0000 $\pm$ 0.0000 \\
      \midrule
      \multirow{2}{*}{U-Net} & \multirow{2}{*}{\underline{0.0000} $\pm$ 0.0000} & Pad & 0.0000 $\pm$ 0.0000 & 0.0000 $\pm$ 0.0000 & \underline{0.0000} $\pm$ 0.0000 & 0.0000 $\pm$ 0.0000 \\
        &
        & Upsample & 0.0000 $\pm$ 0.0000 & 0.0000 $\pm$ 0.0000 & 0.000